In [ ]:
import os
from langchain.chat_models import ChatVertexAI
from langchain.prompts import ChatPromptTemplate
#from langchain.chat_models import ChatOpenAI
## Placeholder for API key setup (if using OpenAI)

In [ ]:

chat = ChatVertexAI(model = "chat-bison",
                   max_output_tokens = 256,
                    temperature = 0.0,
                    top_p = 0.8,
                    top_k = 40,)

chat

In [ ]:
template_string = """Berikut yang berada dalam tripel backtick adalah text percakapan antara dokter dan pasien. \ 
Harap ringkaskan keluhan pasien. Jika tidak ada keluhan, katakan pasien tidak menyebutkan keluhan. \
Dan simpulkan juga apakah dokter memberikan layanan konsultasi dalam percakapan. Jika tidak, katakan dokter tidak memberi konsultasi. \
Dan juga simpulkan penyakit apa yang di diagnosa dokter untuk pasien. \
text: ```{text}```
"""

In [ ]:
prompt_template = ChatPromptTemplate.from_template(template_string)

In [ ]:
prompt_template.messages[0].prompt

In [ ]:
prompt_template.messages[0].prompt.input_variables

In [ ]:
percakapan = """
Pasien: Dok, saya sudah batuk 3 hari ini dan hidung tersumbat  \
Dokter: Apakah ada demam juga?  \
Pasien: Sedikit, kemarin sempat 37,8  \
Dokter: Baik, saya akan resepkan obat batuk dan antihistamin ya  \
Pasien: Boleh minum paracetamol juga, Dok?  \
Dokter: Boleh, 3x1 tablet jika masih demam  \

"""

In [ ]:
konsultasi = """
Pasien: sore \ 
Dokter: Catatan Dari Dokter  Resume :    Diagnosis :    Saran :    Dengan demikian sesi konsultasi ini telah berakhir, terima kasih.. \
"""


In [ ]:
customer_messages = prompt_template.format_messages(text=percakapan)

In [ ]:
customer_messages = prompt_template.format_messages(text=konsultasi)

In [ ]:
customer_response = chat(customer_messages)

### RESPON SCHEMA

Agar data lebih mudah disimpan dalam database, maka dibutuhkan respon dalam bentuk data terstruktur. Untuk itu digunakan fasilitas langchain untuk mendapatkannya.

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
keluhan_schema = ResponseSchema(name = "keluhan", 
                                description = "Apakah penyakit yang dikeluhkan pasien? Buat ringkasannya. Jika tidak ada, katakan tidak ada keluhan.")

konsultasi_schema = ResponseSchema(name = "konsultasi",
                                   description = "Apakah dokter memberikan konsultasi kepada pasien? Jika ya, jawab True. Jika tidak, jawab False")

diagnosa_schema = ResponseSchema(name="diagnosa",
                                 description = "Apa diagnosa / kesimpulan dokter untuk penyakit pasien. Jika ada, ringkaskan. Jika tidak ada, jawab tidak ada diagnosa")

response_schema = [keluhan_schema, konsultasi_schema, diagnosa_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schema)

In [ ]:
format_instructions = output_parser.get_format_instructions()

In [ ]:
print(format_instructions)

In [ ]:
instruction_template = """ Untuk text percakapan antara dokter dan pasiennya, harap ambil informasi berikut:

keluhan: Apakah penyakit yang dikeluhkan pasien? Buat ringkasannya. Jika tidak ada, katakan tidak ada keluhan.

konsultasi: Apakah dokter memberikan konsultasi kepada pasien? Jika ya, jawab True. Jika tidak, jawab False

diagnosa: Apa diagnosa / kesimpulan dokter untuk penyakit pasien. Jika ada, ringkaskan. Jika tidak ada, jawab tidak ada diagnosa

text: {text}

{format_instruction}
"""


In [ ]:
the_prompt = ChatPromptTemplate.from_template(template=instruction_template)

messages = the_prompt.format_messages(text=percakapan,format_instruction=format_instructions)

In [ ]:
the_response = chat(messages)

In [ ]:
print(the_response.content)

In [ ]:
output_dict = output_parser.parse(the_response.content)
output_dict

In [ ]:
output_dict.get("diagnosa")